In [16]:
import pandas as pd

In [17]:
customer = pd.read_csv("Customer_Master.csv")
transaction = pd.read_csv("transactions.csv")

In [18]:
customer.head(2)

,customer_id,age_group,home_location,credit_score,account_age_years,account_type,avg_monthly_income,mobile_banking_user,primary_device,primary_os,primary_browser,avg_transaction_amount,transaction_frequency,employment_status,preferred_transaction_types,international_activity,risk_score
0,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,Samsung Internet,6498.703229,22,Self-employed,"['Loan Payment', 'Internet Bill Payment', 'Wat...",True,30
1,2,26-35,Kathmandu,607,7,Savings,34640.469961,True,Android Tablet,Android,Chrome,3946.766234,6,Employed,"['Loan Payment', 'Branch Deposit', 'QR Payment...",True,33


In [19]:
transaction.head(2)

,transaction_id,customer_id,transaction_date,transaction_type,amount,location,ip_address,device,os,browser,...,credit_score,account_age_years,account_type,avg_monthly_income,mobile_banking_user,transaction_frequency,employment_status,international_activity,risk_score,is_suspicious
0,TXN20241124104326,727,11/24/2024 15:29,Inward Remittance,13925.72,Nawalparasi,103.92.86.139,NaN,NaN,NaN,...,673,1,Current,65911.89284,True,20,Employed,False,24,False
1,TXN20241204130277,539,12/4/2024 5:26,ATM Withdrawal,25037.35,Kaski,27.201.191.54,NaN,NaN,NaN,...,666,11,Current,75126.72458,True,3,Employed,False,19,False


#### Find intersecting columns

In [21]:
duplicate_cols = set(customer.columns).intersection(set(transaction.columns))
duplicate_cols

{'account_age_years',
 'account_type',
 'age_group',
 'avg_monthly_income',
 'credit_score',
 'customer_id',
 'employment_status',
 'home_location',
 'international_activity',
 'mobile_banking_user',
 'risk_score',
 'transaction_frequency'}

In [23]:
duplicate_cols = duplicate_cols - {'customer_id'}
duplicate_cols

{'account_age_years',
 'account_type',
 'age_group',
 'avg_monthly_income',
 'credit_score',
 'employment_status',
 'home_location',
 'international_activity',
 'mobile_banking_user',
 'risk_score',
 'transaction_frequency'}

In [24]:
transaction = transaction.drop(columns= duplicate_cols)

In [25]:
#check duplicate again
duplicate_cols = set(customer.columns).intersection(set(transaction.columns))
duplicate_cols

{'customer_id'}

#### merge these datasets

In [26]:
dataset = customer.merge(transaction, on="customer_id", how="inner")

In [27]:
dataset.head(5)

,customer_id,age_group,home_location,credit_score,account_age_years,account_type,avg_monthly_income,mobile_banking_user,primary_device,primary_os,...,device,os,browser,attempt_sequence,time_of_day,transaction_velocity,status,auth_method,amount_deviation,is_suspicious
0,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,NaN,NaN,NaN,1,Evening,1,Success,Face Recognition,1.493718,False
1,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,NaN,NaN,NaN,1,Evening,1,Success,Password,0.776204,False
2,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,NaN,NaN,NaN,1,Late Night,1,Success,Face Recognition,1.211956,False
3,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,NaN,NaN,NaN,1,Early Morning,4,Success,PIN,1.236164,False
4,1,26-35,Palpa,714,13,Savings,55580.825844,True,iPad,iOS,...,NaN,NaN,NaN,1,Early Morning,1,Success,OTP,1.292338,False


In [28]:
#check for null
dataset.isna().sum()

customer_id                        0
age_group                          0
home_location                      0
credit_score                       0
account_age_years                  0
account_type                       0
avg_monthly_income                 0
mobile_banking_user                0
primary_device                 14202
primary_os                     14202
primary_browser                14202
avg_transaction_amount             0
transaction_frequency              0
employment_status                  0
preferred_transaction_types        0
international_activity             0
risk_score                         0
transaction_id                     0
transaction_date                   0
transaction_type                   0
amount                             0
location                           0
ip_address                         0
device                         79991
os                             79963
browser                        79967
attempt_sequence                   0
t

In [29]:
dataset.shape

(103500, 33)

In [31]:
#device, os and browser have about 77% of null data so we drop those columns
dataset.drop(columns=["device", "os", "browser", "attempt_sequence"], inplace= True)

In [33]:
dataset.duplicated().sum()

np.int64(0)

In [34]:
dataset.isna().sum()

customer_id                        0
age_group                          0
home_location                      0
credit_score                       0
account_age_years                  0
account_type                       0
avg_monthly_income                 0
mobile_banking_user                0
primary_device                 14202
primary_os                     14202
primary_browser                14202
avg_transaction_amount             0
transaction_frequency              0
employment_status                  0
preferred_transaction_types        0
international_activity             0
risk_score                         0
transaction_id                     0
transaction_date                   0
transaction_type                   0
amount                             0
location                           0
ip_address                         0
time_of_day                        0
transaction_velocity               0
status                             0
auth_method                        0
a

In [35]:
#now filling values with mode for 3 remaining columns
dataset["primary_device"].fillna(dataset["primary_device"].mode()[0], inplace= True)
dataset["primary_os"].fillna(dataset["primary_os"].mode()[0], inplace= True)
dataset["primary_browser"].fillna(dataset["primary_browser"].mode()[0], inplace= True)

C:\Users\Sandesh Khatiwada\AppData\Local\Temp\ipykernel_12544\1276973883.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["primary_device"].fillna(dataset["primary_device"].mode()[0], inplace= True)
C:\Users\Sandesh Khatiwada\AppData\Local\Temp\ipykernel_12544\1276973883.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

In [36]:
dataset.isna().sum().sum()

np.int64(0)